In [1]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Project').getOrCreate()

dataset=spark.read.csv("items.tbl", inferSchema = True, header = True, sep = '|')

In [5]:
dataset.createTempView("items")

In [67]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json 

sc = spark.sparkContext
js_dataset=spark.read.json("clicks.json")
df =(js_dataset).to_json()
a = json.dumps(df)
jsonRDD =sc.parallelize(a)

AttributeError: 'DataFrame' object has no attribute 'to_json'

In [33]:
from IPython.display import * 

In [63]:
web_logs = jsonToDataFrame("""
{
  "wl_custom_id": "{\\"wl_item_id\\":1}"
}
""")


Py4JError: An error occurred while calling o709.__getstate__. Trace:
py4j.Py4JException: Method __getstate__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [73]:
from pandas import Series, DataFrame

import pandas as pd

data = []
with open('clicks.json') as f:
    for line in f:
        data.append(json.loads(line))

    df = DataFrame(data, columns=['wl_custom_id', 'wl_item_id'])

In [75]:
print(df)

     wl_custom_id  wl_item_id
0             NaN         NaN
1             NaN       858.0
2             NaN      1567.0
3             NaN      1822.0
4             NaN       193.0
..            ...         ...
200           NaN      2076.0
201           NaN        46.0
202           NaN         NaN
203           NaN       683.0
204           NaN       455.0

[205 rows x 2 columns]


In [79]:
type(df)

pandas.core.frame.DataFrame

In [82]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()
df_1 = spark.createDataFrame(df)
df_1.createTempView("clicks")

In [89]:
q="select c.wl_custom_id, i.i_category_id from items i join clicks c on c.wl_item_id=i.i_item_id where c.wl_custom_id is not NULL"

In [90]:
df = spark.sql(q)

In [91]:
print(df)

DataFrame[wl_custom_id: double, i_category_id: int]


In [95]:
df.createTempView("web_clickstreams")

In [100]:
q2="select wl_custom_id, sum(case when i_category_id=0 then 1 else 0 end) as clicks_in_0,sum(case when i_category_id=1 then 1 else 0 end) as clicks_in_1,sum(case when i_category_id=2 then 1 else 0 end) as clicks_in_2,sum(case when i_category_id=3 then 1 else 0 end) as clicks_in_3,sum(case when i_category_id=4 then 1 else 0 end) as clicks_in_4,sum(case when i_category_id=5 then 1 else 0 end) as clicks_in_5,sum(case when i_category_id=6 then 1 else 0 end) as clicks_in_6,sum(case when i_category_id=7 then 1 else 0 end) as clicks_in_7,sum(case when i_category_id=8 then 1 else 0 end) as clicks_in_8,sum(case when i_category_id=9 then 1 else 0 end) as clicks_in_9,sum(case when i_category_id=10 then 1 else 0 end) as clicks_in_10,sum(case when i_category_id=11 then 1 else 0 end) as clicks_in_11,sum(case when i_category_id=12 then 1 else 0 end) as clicks_in_12,sum(case when i_category_id=13 then 1 else 0 end) as clicks_in_13,sum(case when i_category_id=14 then 1 else 0 end) as clicks_in_14,sum(case when i_category_id=15 then 1 else 0 end) as clicks_in_15,sum(case when i_category_id=16 then 1 else 0 end) as clicks_in_16,sum(case when i_category_id=17 then 1 else 0 end) as clicks_in_17,sum(case when i_category_id=18 then 1 else 0 end) as clicks_in_18,sum(case when i_category_id=19 then 1 else 0 end) as clicks_in_19 from web_clickstreams group by wl_custom_id"

In [115]:
df_2 = spark.sql(q2)
print(df_2)

DataFrame[wl_custom_id: double, clicks_in_0: bigint, clicks_in_1: bigint, clicks_in_2: bigint, clicks_in_3: bigint, clicks_in_4: bigint, clicks_in_5: bigint, clicks_in_6: bigint, clicks_in_7: bigint, clicks_in_8: bigint, clicks_in_9: bigint, clicks_in_10: bigint, clicks_in_11: bigint, clicks_in_12: bigint, clicks_in_13: bigint, clicks_in_14: bigint, clicks_in_15: bigint, clicks_in_16: bigint, clicks_in_17: bigint, clicks_in_18: bigint, clicks_in_19: bigint]


In [102]:
df_2.createTempView("category_clicks")

In [103]:
q3="SELECT CASE WHEN clicks_in_0 > 3.871510156577754 THEN 1.0 ELSE 0.0 END AS label,clicks_in_1,clicks_in_2,clicks_in_3,clicks_in_4,clicks_in_5,clicks_in_6,clicks_in_7,clicks_in_8,clicks_in_9,clicks_in_10,clicks_in_11,clicks_in_12,clicks_in_13,clicks_in_14,clicks_in_15,clicks_in_16,clicks_in_17,clicks_in_18,clicks_in_19 FROM category_clicks"

In [104]:
df_3 =spark.sql(q3)

In [105]:
print(df_3)

DataFrame[label: decimal(2,1), clicks_in_1: bigint, clicks_in_2: bigint, clicks_in_3: bigint, clicks_in_4: bigint, clicks_in_5: bigint, clicks_in_6: bigint, clicks_in_7: bigint, clicks_in_8: bigint, clicks_in_9: bigint, clicks_in_10: bigint, clicks_in_11: bigint, clicks_in_12: bigint, clicks_in_13: bigint, clicks_in_14: bigint, clicks_in_15: bigint, clicks_in_16: bigint, clicks_in_17: bigint, clicks_in_18: bigint, clicks_in_19: bigint]


In [108]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["clicks_in_1", "clicks_in_2", "clicks_in_3", "clicks_in_4", "clicks_in_5", "clicks_in_6" , "clicks_in_7", "clicks_in_8", "clicks_in_9" , "clicks_in_10", "clicks_in_11", "clicks_in_12" , "clicks_in_13", "clicks_in_14", "clicks_in_15" , "clicks_in_16", "clicks_in_17", "clicks_in_18" , "clicks_in_19"],outputCol ="features") 
vd = assembler.transform(df_3)

In [114]:
print(vd)

DataFrame[label: decimal(2,1), clicks_in_1: bigint, clicks_in_2: bigint, clicks_in_3: bigint, clicks_in_4: bigint, clicks_in_5: bigint, clicks_in_6: bigint, clicks_in_7: bigint, clicks_in_8: bigint, clicks_in_9: bigint, clicks_in_10: bigint, clicks_in_11: bigint, clicks_in_12: bigint, clicks_in_13: bigint, clicks_in_14: bigint, clicks_in_15: bigint, clicks_in_16: bigint, clicks_in_17: bigint, clicks_in_18: bigint, clicks_in_19: bigint, features: vector]


In [111]:
training, test = vd.randomSplit([0.6, 0.4], seed)

In [112]:
layers = [19, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(training)

In [113]:
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="presicion")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = nan
